In [1]:
#Import Packages
from selenium import webdriver
from selenium.webdriver.common.by import By
from dotenv import load_dotenv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import os
import time

In [2]:
dashboardUrl = "https://lms.lpuonline.com/"

In [3]:
#Load the web driver and get the url
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(dashboardUrl)

In [4]:
load_dotenv()
username = os.environ.get("registrationId")
password = os.environ.get("password")
print(type(username), type(password))

<class 'str'> <class 'str'>


In [5]:
usernameInput = driver.find_element(By.ID, "username")
passwordInput = driver.find_element(By.ID, "Password")
loginButton = driver.find_element(By.ID, "LoginButton")

In [6]:
usernameInput.send_keys(username)
passwordInput.send_keys(password)
loginButton.click()

In [7]:
totalSubjects = driver.find_element(By.ID, "crprogress").find_elements(By.CLASS_NAME, "prog-card").__len__()

In [8]:
def lectureHandler(inPlay = False):
    totalLectures = (driver.find_element(By.CLASS_NAME, "inPlay") if inPlay else driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "collapse")).find_elements(By.CLASS_NAME, "videoli").__len__()
    if totalLectures == 0:
        raise Exception("Lectures not loaded") # raise error if lectures are not found
    
    for currentLectureIndex in range(totalLectures):
        print("currentLectureIndex ", currentLectureIndex, "total lectures in current unit is ", totalLectures)
        
        progressText = driver.find_element(By.CLASS_NAME, "inPlay").find_element(By.CLASS_NAME, "progress-text").text
        if inPlay and progressText == "100%": # exit from this function if inPlay = True and all lectures inPlay are already watched
            print("100% Video Progress")
            break 
        
        try: # try needed because when 1st video of current unit completed then current unit lectures are shifted to inPlay div 
            nextLecture =  (driver.find_element(By.CLASS_NAME, "inPlay") if inPlay else driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "collapse")).find_elements(By.CLASS_NAME, "videoli")[currentLectureIndex]
        except: # if nextLecture raise indexing error then take nextLecture will be taken from inPlay div
            nextLecture = driver.find_element(By.CLASS_NAME, "inPlay").find_elements(By.CLASS_NAME, "videoli")[currentLectureIndex]
        
        nextLecture.find_element(By.CLASS_NAME, "cc-a").click()
        time.sleep(10) # wait for player initiate
        driver.execute_script(open("./videoHandler.js").read())
        time.sleep(10)

In [10]:
for currentSubjectIndex in range(totalSubjects):
    print("\n\ncurrentSubjectIndex ", currentSubjectIndex)
    # click find and click on currentSubjectCard
    allSubjects = driver.find_element(By.ID, "crprogress").find_elements(By.CLASS_NAME, "prog-card")
    allSubjects[currentSubjectIndex].find_element(By.TAG_NAME, 'a').click()
    
    totalRemainingUnits = driver.find_element(By.CLASS_NAME, "unit").find_elements(By.CLASS_NAME, "collapse").__len__()
    print("\ntotalRemainingUnit", totalRemainingUnits)
    
    # wait for load videoli inside inPlay div to play all lectures inside inPlay div
    WebDriverWait(driver, 30).until(lambda driver: expected_conditions.presence_of_element_located((By.CLASS_NAME, "videoli"))(driver.find_element(By.CLASS_NAME, "inPlay"))) 
    lectureHandler(True) # pass True because here we just want to play all lectures inside inPlay div
    
    currentRemainingUnitIndex = 1
    while currentRemainingUnitIndex <= totalRemainingUnits:
        print("\ncurrentUnitIndex ", currentRemainingUnitIndex)
        # get current unit title element and wait for element to be clickable
        currentUnitTitle = driver.find_element(By.CLASS_NAME, "unit").find_element(By.CLASS_NAME, "lecture-title")
        WebDriverWait(driver, 10).until(expected_conditions.element_to_be_clickable(currentUnitTitle))
        
        currentUnitTitle.click() # click title element and wait for load lectures inside that unit's collapsed div
        WebDriverWait(driver, 30).until(lambda driver: expected_conditions.presence_of_element_located((By.CLASS_NAME, "videoli"))(currentUnitTitle.find_element(By.XPATH, "./following-sibling::div")))
        
        try:
           lectureHandler()
        except Exception as e:
            if e.args[0] == "Lectures not loaded":
                print(e, " in current unit on first try so wait trying again")
                WebDriverWait(driver, 30).until(lambda driver: expected_conditions.presence_of_element_located((By.CLASS_NAME, "videoli"))(currentUnitTitle.find_element(By.XPATH, "./following-sibling::div")))
                try:
                    lectureHandler()
                except Exception as e:
                    print(e, " on second try. In", currentUnitTitle.find_element(By.CLASS_NAME, "col-4").text)
        # increment currentRemainingUnitIndex to go on next unit
        currentRemainingUnitIndex = currentRemainingUnitIndex + 1
        
    # goto home page when script completely go through all lectures
    driver.back()

print("\nAll Good :D") # print all good when script executed without any unexpected error



currentSubjectIndex  0

totalRemainingUnit 0
currentLectureIndex  0 total lectures in current unit is  3
100% Video Progress


currentSubjectIndex  1

totalRemainingUnit 0
currentLectureIndex  0 total lectures in current unit is  1
100% Video Progress


currentSubjectIndex  2

totalRemainingUnit 0
currentLectureIndex  0 total lectures in current unit is  2
100% Video Progress


currentSubjectIndex  3

totalRemainingUnit 0
currentLectureIndex  0 total lectures in current unit is  10
100% Video Progress


currentSubjectIndex  4

totalRemainingUnit 0
currentLectureIndex  0 total lectures in current unit is  1
100% Video Progress


currentSubjectIndex  5

totalRemainingUnit 1
currentLectureIndex  0 total lectures in current unit is  3
100% Video Progress

currentUnitIndex  1
currentLectureIndex  0 total lectures in current unit is  4
currentLectureIndex  1 total lectures in current unit is  4
currentLectureIndex  2 total lectures in current unit is  4
currentLectureIndex  3 total lecture